# DiffQRCoder Pipeline Notebook

This notebook generates QR codes from URLs and creates artistic QR codes using DiffQRCoder.

In [ ]:
import subprocess
import sys
import os

# Ensure output directories exist
os.makedirs("qrcodes", exist_ok=True)
os.makedirs("outputs", exist_ok=True)

## Step 1: Generate QR Code from URL

In [ ]:
def generate_qr_from_url(url, output_path="qrcodes", filename=None):
    """
    Generate a QR code from URL using the plain_code_from_url.py script.
    """
    args = [sys.executable, "DiffQRCoder/plain_code_from_url.py", url, "--output-path", output_path]
    
    if filename:
        args.extend(["--filename", filename])
    
    result = subprocess.run(args, capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ QR code generated successfully!")
        print(result.stdout)
    else:
        print(f"❌ Error generating QR code (code {result.returncode})")
        print(result.stderr)
    
    return result

# Generate QR code
url = "https://example.com/your-url-here"
qr_result = generate_qr_from_url(url)

## Step 2: Create Artistic QR Code with DiffQRCoder

In [ ]:
def run_diffqrcoder(prompt, qrcode_path, output_path, 
                   qrcode_module_size=20, 
                   controlnet_conditioning_scale=1.05,
                   scanning_robust_guidance_scale=50,
                   perceptual_guidance_scale=20):
    """
    Run DiffQRCoder with specified parameters using subprocess.
    """
    args = [
        sys.executable, "DiffQRCoder/run_diffqrcoder.py",
        "--prompt", prompt,
        "--qrcode_path", qrcode_path,
        "--qrcode_module_size", str(qrcode_module_size),
        "--controlnet_conditioning_scale", str(controlnet_conditioning_scale),
        "--scanning_robust_guidance_scale", str(scanning_robust_guidance_scale),
        "--perceptual_guidance_scale", str(perceptual_guidance_scale),
        "--output_path", output_path
    ]
    
    result = subprocess.run(args, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Success! Artistic QR code saved to: {output_path}")
        print(result.stdout)
    else:
        print(f"❌ Error (code {result.returncode})")
        print(result.stderr)
    
    return result

# Create artistic QR code
diffqr_result = run_diffqrcoder(
    prompt="A healthy berry muffin",
    qrcode_path="qrcodes/example.com_your-url-here.png",  # Update this based on your URL
    output_path="outputs/muffin_qrcode.png"
)

## Complete Pipeline Function

In [ ]:
def create_artistic_qr_pipeline(url, prompt, output_filename):
    """
    Complete pipeline: Generate QR code from URL, then create artistic version.
    """
    print(f"🔗 Processing URL: {url}")
    print(f"🎨 Prompt: {prompt}")
    print("-" * 50)
    
    # Step 1: Generate basic QR code
    qr_result = generate_qr_from_url(url)
    if qr_result.returncode != 0:
        return False
    
    # Extract the generated filename from the URL
    from urllib.parse import urlparse
    parsed = urlparse(url)
    qr_filename = (parsed.netloc + parsed.path).replace('/', '_').replace('\\', '_').rstrip('_') + '.png'
    qr_path = f"qrcodes/{qr_filename}"
    
    print(f"\n🎭 Creating artistic QR code...")
    
    # Step 2: Create artistic QR code
    diffqr_result = run_diffqrcoder(
        prompt=prompt,
        qrcode_path=qr_path,
        output_path=f"outputs/{output_filename}"
    )
    
    return diffqr_result.returncode == 0

# Example usage
success = create_artistic_qr_pipeline(
    url="https://github.com/your-repo",
    prompt="A futuristic digital landscape with neon lights",
    output_filename="github_artistic_qr.png"
)

if success:
    print("\n🎉 Pipeline completed successfully!")
else:
    print("\n💥 Pipeline failed!")